In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import os
# import json
# import time
# import pandas as pd
# import random
# from openai import OpenAI, RateLimitError
# from tenacity import retry, wait_random_exponential, stop_after_attempt
# from datasets import load_dataset
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# # --- 1. Setup and API Call Functions ---

# openai_key = user_secrets.get_secret("openai")
# if not openai_key:
#     raise ValueError("OPENAI_API_KEY environment variable not set.")
# client = OpenAI(api_key=openai_key)


# def call_gpt4o_mini(messages):
#     """Sends a prompt to the GPT-4o-mini API with robustness for errors."""
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages,
#             response_format={"type": "json_object"},
#             temperature=0.7,
#             n=1
#         )
#         content = response.choices[0].message.content
#         return json.loads(content)
#     except RateLimitError as e:
#         print(f"Rate limit error: {e}. Retrying...")
#         raise
#     except json.JSONDecodeError as e:
#         print(f"Received invalid JSON from API: {e}. Retrying...")
#         raise e
#     except Exception as e:
#         print(f"An unexpected error occurred during the API call: {e}")
#         raise e

# # --- 2. Data Loading and Filtering ---

# def load_and_filter_scienceqa_data(subset_name="natural science"):
#     """
#     Loads the ScienceQA dataset, filters for a specific subset, and then
#     further filters for text-only questions with a valid lecture/solution context.
#     """
#     print("Loading ScienceQA dataset...")
#     ds = load_dataset("derek-thomas/ScienceQA")
#     df = ds['train'].to_pandas()
#     df = df[df['subject'] == subset_name].iloc[:50]

#     text_only_df = df[df['image'].isnull()].copy()

#     def build_context(row):
#         context_parts = []
#         if pd.notna(row['lecture']) and len(str(row['lecture'])) > 0:
#             context_parts.append(str(row['lecture']))
#         if pd.notna(row['solution']) and len(str(row['solution'])) > 0:
#             context_parts.append(str(row['solution']))
#         if pd.notna(row['hint']) and len(str(row['hint'])) > 0:
#             # Use 'Required Context:' to explicitly tell the model this is important
#             context_parts.append('Required Context: ' + str(row['hint']))
#         return ' '.join(context_parts)

#     text_only_df['text_context'] = text_only_df.apply(build_context, axis=1)

#     # Filter out rows where the combined context is empty
#     text_only_df = text_only_df[text_only_df['text_context'].str.len() > 0]

#     print(f"Loaded and filtered {len(text_only_df)} text-only questions.")
#     return text_only_df

# # --- 3. Revised Single-Prompt Function with Constraints ---

# def construct_single_stage_prompt(context):
#     """
#     Constructs a single prompt to generate a question, options, and distractors
#     with strong constraints for thematic coherence and grounding.
#     """
#     user_content = f"""
#     You are an expert at creating high-quality, complex multiple-choice questions.

#     Your task is to generate one question based on the provided context. The correct answer must be a single option that combines two or more facts using a logical operator ("and" or "or"). The distractors must be plausible but factually incorrect.

#     Let's think step by step:
#     1.  Read the context carefully and identify a specific thematic category (e.g., "states of matter," "living organisms").
#     2.  If the context contains "**Required Context:**", this information is critical.
#     3.  From the context, extract a set of independent, atomic facts. Categorize these as either "Correct Facts" or "Incorrect Facts" (plausible but wrong claims).
#     4.  Select a subset of the "Correct Facts" and combine them into a single, compound correct option using "and" or "or."
#     5.  Formulate a concise question that relies on this combined logic.
#     6.  Create 4-6 options in total. The correct option will be one of these. The distractors can be either "Incorrect Facts" or compound statements made from mixing correct and incorrect facts.
#     7.  Randomly assign a letter (A, B, C, D, E, F) to the correct option. 
#     8.  Format the final output as a single JSON object.

#     Strict Constraints:
#     -   **Fact-Grounded:** The entire question, all facts, and all options must be strictly derived from the context.
#     -   **No Pronouns:** All options must use proper subject names or descriptive phrases (e.g., "solids" instead of "they").
#     -   **No Repetition:** The correct option MUST be a novel combination of facts. It must NOT be a direct copy of a compound statement from the context.
#     -   **Randomized Correct Answer:** The `correct_answer_label` must be randomly assigned and not always 'A'.
#     -   **Multiple Options:** Generate 4-6 options.
#     -   **JSON Format:** The final output must be a single JSON object with the following keys: `correct_facts`, `incorrect_facts`, `question`, `choices`, `correct_answer_label`, and `logical_structure`.

#     Context:
#     {context}

#     Output JSON Example:
#     {{
#       "correct_facts": [
#         "Solids have a definite shape.",
#         "Solids have a definite volume.",
#         "Liquids take the shape of their container."
#       ],
#       "incorrect_facts": [
#         "Solids can be bent easily.",
#         "Liquids have a definite shape."
#       ],
#       "question": "Which of the following describes the key characteristics of solids and liquids?",
#       "choices": [
#         "A. Solids have a definite shape and liquids have a definite volume.",
#         "B. Liquids have a definite shape and Liquids have a definite volume.",
#         "C. Solids take the shape of their container or solida hhave a definite volume.",
#         "D. Solids have no definite shape."
#       ],
#       "correct_answer_label": "A",
#       "logical_structure": "AND"
#     }}
    
#     {{
#   "correct_facts": [
#     "Solar energy is a renewable resource.",
#     "Wind energy is a renewable resource.",
#     "Solar panels convert sunlight into electricity.",
#     "Wind turbines convert wind into electricity."
#   ],
#   "incorrect_facts": [
#     "Solar energy is a nonrenewable resource.",
#     "Wind turbines convert coal into electricity.",
#     "Solar panels generate energy at night."
#   ],
#   "question": "Which of the following describes a true statement about renewable energy generation?",
#   "choices": [
#     "A. Wind energy is a nonrenewable resource or wind turbines convert coal into electricity.",
#     "B. Solar energy is a nonrenewable resource and solar panels convert sunlight into electricity.",
#     "C. Solar panels convert sunlight into electricity or wind turbines convert wind into electricity.",
#     "D. Solar panels generate energy at night."
#   ],
#   "correct_answer_label": "C",
#   "logical_structure": "OR"
# }}
#     """
#     messages = [
#         {"role": "system", "content": "You are a QA system that strictly follows instructions."},
#         {"role": "user", "content": user_content}
#     ]
#     return messages

# # --- 4. Main Execution Block ---
# def process_single_stage_pipeline(df, output_file_csv):
#     """
#     Orchestrates the single-stage pipeline and saves the final output to a CSV file.
#     """
#     print(f"Starting single-stage pipeline for {len(df)} rows.")
#     final_output_list = []
    
#     for i in range(len(df)):
#         row = df.iloc[i]
        
#         # This is where the model generates the question
#         messages = construct_single_stage_prompt(row['text_context'])
#         try:
#             result = call_gpt4o_mini(messages)
            
#             # This is where we add the metadata to the final output
#             result['original_context'] = row['text_context']
            
#             # Add the hint to the final output if it exists in the original row
#             if pd.notna(row['hint']) and len(str(row['hint'])) > 0:
#                 result['original_hint'] = str(row['hint'])

#             final_output_list.append(result)
#             print(f"  Processed row {i} successfully.")
#         except Exception as e:
#             print(f"  Failed to process row {i}. Skipping.")
#         time.sleep(0.01)

#     final_df = pd.DataFrame(final_output_list)
#     final_df.to_csv(output_file_csv, index=False, encoding='utf-8')
#     print(f"\nProcessing complete. {len(final_output_list)} final results saved to '{output_file_csv}'.")

# if __name__ == '__main__':
#     filtered_df = load_and_filter_scienceqa_data()
#     if not filtered_df.empty:
#         output_csv_file = 'scienceqa_single_stage_generated.csv'
#         process_single_stage_pipeline(filtered_df, output_csv_file)

In [3]:
# results = pd.read_csv('scienceqa_single_stage_generated.csv')

In [4]:
# results

In [5]:
# def print_final_outputs(df):
#     """Prints the final generated questions in a clean, readable format."""
#     print("\n" + "="*50)
#     print("      SAMPLE GENERATED QUESTIONS")
#     print("="*50 + "\n")
    
#     for idx, row in df.iterrows():
#         print(f"--- Question {idx+1} ---")
#         print(f"  Logical Structure: {row.get('logical_structure')}")
#         print(f"  Question: {row.get('question')}")

#         print()
#         print(f"Correct Facts: ",{row['correct_facts']})
#         print(f"Incorrect Facts: ",{row['incorrect_facts']})   
#         print()
#         # Print choices
#         choices = row.get('choices')
#         if isinstance(choices, str):
#             print("  Choices:")
#             print(choices)
        
#         print(f"  Correct Answer Label: {row.get('correct_answer_label')}")
        
#         # Conditionally print the original hint if it exists
#         if 'original_hint' in row and pd.notna(row['original_hint']):
#             print(f"  Original Hint: {row['original_hint']}")

#         print("\n" + "-"*40 + "\n")

In [6]:
# print_final_outputs(results)

In [7]:
import os
import json
import time
import pandas as pd
import random
from openai import OpenAI, RateLimitError
from tenacity import retry, wait_random_exponential, stop_after_attempt
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

# --- 1. Setup and API Call Functions ---

openai_key = user_secrets.get_secret("openai")
if not openai_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")
client = OpenAI(api_key=openai_key)


def call_gpt4o_mini(messages):
    """Sends a prompt to the GPT-4o-mini API with robustness for errors."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.5,
            n=1
        )
        content = response.choices[0].message.content
        return json.loads(content)
    except RateLimitError as e:
        print(f"Rate limit error: {e}. Retrying...")
        raise
    except json.JSONDecodeError as e:
        print(f"Received invalid JSON from API: {e}. Retrying...")
        raise e
    except Exception as e:
        print(f"An unexpected error occurred during the API call: {e}")
        raise e

# --- 2. Data Loading and Filtering ---

def load_and_filter_scienceqa_data(subset_name="natural science"):
    """
    Loads the ScienceQA dataset, filters for a specific subset, and then
    further filters for text-only questions with a valid lecture/solution context.
    """
    print("Loading ScienceQA dataset...")
    ds = load_dataset("tau/commonsense_qa")
    train_df = ds['train'].to_pandas()
    valid_df  = ds['validation'].to_pandas()
    test_df = ds['test'].to_pandas()
    df = pd.concat([train_df, valid_df, test_df], ignore_index=True)
    print(df.shape)
    
    print(f"Loaded and filtered {len(df)} text-only questions.")
    return df.iloc[:5000]

# # --- 3. Revised Single-Prompt Function with Constraints ---
# def construct_single_stage_options_prompt(question, correct_answers, incorrect_answers, context=""):
#     """
#     Single-stage fallback: directly produce AND/OR options with multi-level subclaims.
#     Less controllable than two-stage, but convenient.
#     """
#     user_content = f"""
# You will produce multi-step AND/OR options for a multiple-choice item.

# INPUT
# - Question: {question}
# - Correct Atomic Facts: {correct_answers}
# - Incorrect Atomic Facts: {incorrect_answers}
# - Optional background (do NOT reference explicitly): {context}

# REQUIREMENTS
# - Produce >=2 correct and 3–4 incorrect options.
# - Each option must be exactly: <Subclaim A> AND/OR <Subclaim B>.
# - Each Subclaim (A, B) must itself be multi-level by merging ≥2 correct atomic facts (for correct options).
# - Remember each subclain have to be multi-step and multi-level where one has to reason over the question and the answer to get to the correct answer.
# - Incorrect options must integrate at least one incorrect atomic fact (or misuse AND/OR) to create a plausible but false composite.
# - No “based on the context” phrasing. No copy/paste of facts.

# OUTPUT (JSON only):
# {{
#   "question": "<repeat question>",
#   "correct_combinations": ["...", "..."],
#   "incorrect_combinations": ["...", "...", "..."],
#   "notes_private": "Very brief note on how each subclaim merged multiple facts and where a flaw was introduced."
# }}
# Return JSON only.
# """
#     messages = [
#         {"role": "system", "content": "You are a QA system that composes multi-step options and returns strict JSON only."},
#         {"role": "user", "content": user_content}
#     ]
#     return messages

def generate_robust_logical_options(question, existing_correct, existing_incorrect, context=""):
    user_content = f"""
You are a cognitive and logical reasoning expert, skilled in designing assessments that test authentic, multi-step inference. Your specialty is crafting challenging options that force high school students with deep understanding to demonstrate genuine causal understanding rather than surface-level pattern matching. The goal is creating options that require connecting multiple concepts through valid causal chains.

QUESTION: {question}
EXISTING CORRECT: {existing_correct}
EXISTING INCORRECT: {existing_incorrect}

TASK: Expand and improve the existing option pools using the following instructions, making the selection challenging even for high school students.

ADDITIONAL CORRECT OPTIONS Rubric (Generate 4-6 more):
- Use word action phrases or method descriptions
- Should complement existing correct options (different approaches/mechanisms)
- Each must require multi-step causal reasoning to evaluate
- NO explanations of why they work 
- Test genuine understanding, not keyword matching

ADDITIONAL INCORRECT OPTIONS Rubric (Generate 4-6 more):
- Include deceptive and plausible wrong answers
- Should complement existing incorrect options
- Mix of: contextually wrong, mechanism failures, obvious contradictions

DISTRACTOR DIFFICULTY LEVELS:
- 50% Near-miss options: Work in similar contexts but fail due to subtle mechanism differences
- 30% Partial truth options: Contain correct elements but missing critical components  
- 20% Edge case failures: Seem reasonable but fail under specific constraints

DECEPTIVE QUALITY PATTERNS:
- Use domain-appropriate terminology that sounds expert-level
- Include options that work for related but different goals
- Create options where timing/context makes them invalid
- Generate options that have worked historically but don't work now
- Include options where the mechanism is partially correct but incomplete
- You may include options that fail at a later step of reasoning. (Recommended)

AVOID OBVIOUS GIVEAWAYS:
- No clearly wrong domains ("gym" for "finding people outdoors")
- No obviously impossible actions ("flying" for human travel)
- No silly/joke options that immediately stand out
- No options that contradict basic science or common sense
- Avoid shallow wrong answers that can be rejected with a keyword check.

QUALITY REQUIREMENTS:
- Each option must force: Multi Action → Multi Effect → Goal Achievement reasoning
- Include edge cases that test deep vs surface understanding.
- Test causal mechanism understanding, not just correlation.
- Each option must require connecting more than 2 distinct concepts to evaluate.
- You can include constraints that make plausible options actually invalid.
- You may include options requiring understanding real-world context and underlying implicit goals.
- Options in the same bin shouldn't mean the same.
- The answers should be directly factual, instead force the reader to think and reason deeply. (Most IMPORTANT)
- WORD LIMIT: Keep all new options the same length or shorter than the longest exisitng option given to you. (IMPORTANT)
- Try to maintain the same word length for both correct and incorrect new options.

FORBIDDEN PATTERNS:
- Shouldn't contain explanatory phrases
- Shouldn't include location descriptions ("park with scenic view" → just "park")
- Shouldn't use conjunctions within options ("and", "but", "or" within single option)
- Shouldn't provide reasoning ("because", "since", "due to")

OUTPUT (JSON):
{{
  "question": "{question}",
  "expanded_correct_options": [
    "existing correct option 1",
    "existing correct option 2", 
    "new correct option 1",
    "new correct option 2",
    "new correct option 3"
  ],
  "expanded_incorrect_options": [
    "existing wrong option",
    "existing wrong option",
    "plausible wrong option",
    "contextual wrong option", 
    "mechanism failure option",
    "obvious wrong option",
    "plausible wrong option",
    "contextual wrong option",
    "mechanism failure option", 
    "obvious wrong option"
  ]
}}

Create a more comprehensive and challenging set.
"""
    messages = [
        {
            "role": "system", 
            "content": "You are a cognitive and logical reasoning expert, skilled in designing assessments that test authentic, multi-step inference. Your specialty is crafting challenging options that force high school students with deep understanding to demonstrate genuine causal understanding rather than surface-level pattern matching. The goal is creating options that require connecting multiple concepts through valid causal chains. Each option must be evaluable independently. Return only valid JSON."
        },
        {
            "role": "user", 
            "content": user_content
        }
    ]
    return messages




# --- 4. Main Execution Block ---
def process_single_stage_pipeline(df, output_file_csv):
    """
    Orchestrates the single-stage pipeline and saves the final output to a CSV file.
    """
    print(f"Starting single-stage pipeline for {len(df)} rows.")
    final_output_list = []
    
    for i in range(len(df)):
        row = df.iloc[i]
        question = row['question']
        labels = row['choices']['label']
        texts = row['choices']['text']
        
        correct_label = row['answerKey']  # Assuming this exists
        correct_answer = texts[labels == correct_label][0]
        
        # Get incorrect answers
        incorrect_answers = texts[labels != correct_label]
        
        # This is where the model generates the question
        messages = generate_robust_logical_options(question, correct_answer, incorrect_answers)
        try:
            result = call_gpt4o_mini(messages)
            
            # This is where we add the metadata to the final output
            # result['original_context'] = row['text_context']
            
            # Add the hint to the final output if it exists in the original row
            # if pd.notna(row['hint']) and len(str(row['hint'])) > 0:
                # result['original_hint'] = str(row['hint'])

            final_output_list.append(result)
            print(f"  Processed row {i} successfully.")
        except Exception as e:
            print(f"  Failed to process row {i}. Skipping.")
        time.sleep(0.01)

    final_df = pd.DataFrame(final_output_list)
    final_df.to_csv(output_file_csv, index=False, encoding='utf-8')
    print(f"\nProcessing complete. {len(final_output_list)} final results saved to '{output_file_csv}'.")

if __name__ == '__main__':
    filtered_df = load_and_filter_scienceqa_data()
    if not filtered_df.empty:
        output_csv_file = 'csqa_first_stage-5000.csv'
        process_single_stage_pipeline(filtered_df, output_csv_file)

Loading ScienceQA dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

(12102, 5)
Loaded and filtered 12102 text-only questions.
Starting single-stage pipeline for 5000 rows.
  Processed row 0 successfully.
  Processed row 1 successfully.
  Processed row 2 successfully.
  Processed row 3 successfully.
  Processed row 4 successfully.
  Processed row 5 successfully.
  Processed row 6 successfully.
  Processed row 7 successfully.
  Processed row 8 successfully.
  Processed row 9 successfully.
  Processed row 10 successfully.
  Processed row 11 successfully.
  Processed row 12 successfully.
  Processed row 13 successfully.
  Processed row 14 successfully.
  Processed row 15 successfully.
  Processed row 16 successfully.
  Processed row 17 successfully.
  Processed row 18 successfully.
  Processed row 19 successfully.
  Processed row 20 successfully.
  Processed row 21 successfully.
  Processed row 22 successfully.
  Processed row 23 successfully.
  Processed row 24 successfully.
  Processed row 25 successfully.
  Processed row 26 successfully.
  Processed row

In [8]:
df = pd.read_csv('/kaggle/working/csqa_first_stage-5000.csv')
df.head()

,question,expanded_correct_options,expanded_incorrect_options
0,The sanctions against the school were a punish...,"['ignore', 'overlook', 'diminish', 'undermine'...","['enforce', 'authoritarian', 'yell at', 'avoid..."
1,Sammy wanted to go to where the people were. ...,"['populated areas', 'gathering events', 'commu...","['race track', 'the desert', 'apartment', 'roa..."
2,To locate a choker not located in a jewelry bo...,"['jewelry store', 'thrift shop', 'vintage mark...","['neck', 'jewelry box', 'jewelry box', 'boutiq..."
3,Google Maps and other highway and street GPS s...,"['atlas', 'manual navigation techniques', 'pri...","['united states', 'mexico', 'countryside', 'oc..."
4,"The fox walked from the city into the forest, ...","['natural habitat', 'suitable prey', 'food sou...","['pretty flowers.', 'hen house', 'storybook', ..."


In [9]:
print("\n" + "="*50)
print("      SAMPLE GENERATED QUESTIONS")
print("="*50 + "\n")

for idx, row in df.iloc[:50].iterrows():
    print(f"--- Question {idx+1} ---")
    # print(f"  Logical Structure: {row.get('logical_structure')}")
    print(f"  Question: {row.get('question')}")

    print()
    print(f"Correct Facts: ",{row['expanded_correct_options']})
    print(f"Incorrect Facts: ",{row['expanded_incorrect_options']})   
    print()
    # # Print choices
    # choices = row.get('choices')
    # if isinstance(choices, str):
    #     print("  Choices:")
    #     print(choices)
    
    # print(f"  Correct Answer Label: {row.get('correct_answer_label')}")
    
    # Conditionally print the original hint if it exists
    if 'original_hint' in row and pd.notna(row['original_hint']):
        print(f"  Original Hint: {row['original_hint']}")

    print("\n" + "-"*40 + "\n")


      SAMPLE GENERATED QUESTIONS

--- Question 1 ---
  Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?

Correct Facts:  {"['ignore', 'overlook', 'diminish', 'undermine', 'invalidate', 'negate']"}
Incorrect Facts:  {"['enforce', 'authoritarian', 'yell at', 'avoid', 'accelerate', 'facilitate', 'reinforce', 'support', 'enhance', 'promote']"}


----------------------------------------

--- Question 2 ---
  Question: Sammy wanted to go to where the people were.  Where might he go?

Correct Facts:  {"['populated areas', 'gathering events', 'community festivals', 'busy marketplaces', 'concert venues', 'public transportation hubs']"}
Incorrect Facts:  {"['race track', 'the desert', 'apartment', 'roadblock', 'isolated cabins', 'abandoned warehouses', 'empty beaches', 'remote hiking trails', 'quiet libraries', 'secluded gardens']"}


----------------------------------------

--- Question 3 ---
  Question: To lo